In [ ]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load data
data = pd.read_excel("tournaments_updated.xlsx")

m = gp.Model("Maximizing Entered Tournaments")

x11 = m.addVar(vtype=GRB.INTEGER, name="first_in_ATP_250", lb = 0)
x12 = m.addVar(vtype=GRB.INTEGER, name="first_in_ATP_500", lb = 0)
x13 = m.addVar(vtype=GRB.INTEGER, name="first_in_Grand_Slam", lb = 0)
x14 = m.addVar(vtype=GRB.INTEGER, name="first_in_ATP_1000", lb = 0)
x21 = m.addVar(vtype=GRB.INTEGER, name="second_in_ATP_250", lb = 0)
x22 = m.addVar(vtype=GRB.INTEGER, name="second_in_ATP_500", lb = 0)
x23 = m.addVar(vtype=GRB.INTEGER, name="second_in_Grand_Slam", lb = 0)
x24 = m.addVar(vtype=GRB.INTEGER, name="second_in_ATP_1000", lb = 0)
x41 = m.addVar(vtype=GRB.INTEGER, name="semi_in_ATP_250", lb = 0)
x42 = m.addVar(vtype=GRB.INTEGER, name="semi_in_ATP_500", lb = 0)
x43 = m.addVar(vtype=GRB.INTEGER, name="semi_in_Grand_Slam", lb = 0)
x44 = m.addVar(vtype=GRB.INTEGER, name="semi_in_ATP_1000", lb = 0)
x81 = m.addVar(vtype=GRB.INTEGER, name="quarter_in_ATP_250", lb = 0)
x82 = m.addVar(vtype=GRB.INTEGER, name="quarter_in_ATP_500", lb = 0)
x83 = m.addVar(vtype=GRB.INTEGER, name="quarter_in_Grand_Slam", lb = 0)
x84 = m.addVar(vtype=GRB.INTEGER, name="quarter_in_ATP_1000", lb = 0)
x161 = m.addVar(vtype=GRB.INTEGER, name="round16_in_ATP_250", lb = 0)
x162 = m.addVar(vtype=GRB.INTEGER, name="round16_in_ATP_500", lb = 0)
x163 = m.addVar(vtype=GRB.INTEGER, name="round16_in_Grand_Slam", lb = 0)
x164 = m.addVar(vtype=GRB.INTEGER, name="round16_in_ATP_1000", lb = 0)
x321 = m.addVar(vtype=GRB.INTEGER, name="round32_in_ATP_250", lb = 0)
x322 = m.addVar(vtype=GRB.INTEGER, name="round32_in_ATP_500", lb = 0)
x323 = m.addVar(vtype=GRB.INTEGER, name="round32_in_Grand_Slam", lb = 0)
x324 = m.addVar(vtype=GRB.INTEGER, name="round32_in_ATP_1000", lb = 0)
x641 = m.addVar(vtype=GRB.INTEGER, name="round64_in_ATP_250", lb = 0)
x642 = m.addVar(vtype=GRB.INTEGER, name="round64_in_ATP_500", lb = 0)
x643 = m.addVar(vtype=GRB.INTEGER, name="round64_in_Grand_Slam", lb = 0)
x644 = m.addVar(vtype=GRB.INTEGER, name="round64_in_ATP_1000", lb = 0)
x1281 = m.addVar(vtype=GRB.INTEGER, name="round128_in_ATP_250", lb = 0)
x1282 = m.addVar(vtype=GRB.INTEGER, name="round128_in_ATP_500", lb = 0)
x1283 = m.addVar(vtype=GRB.INTEGER, name="round128_in_Grand_Slam", lb = 0)
x1284 = m.addVar(vtype=GRB.INTEGER, name="round128_in_ATP_1000", lb = 0)
y = {}
for i in range(1, 48):
    y[i] = m.addVar(vtype=GRB.BINARY, name=f"tourney{i}")
# y1 = m.addVar(vtype=GRB.BINARY, name="tourney1")
# y2 = m.addVar(vtype=GRB.BINARY, name="tourney2")
# y3 = m.addVar(vtype=GRB.BINARY, name="tourney3")
# y4 = m.addVar(vtype=GRB.BINARY, name="tourney4")
# y5 = m.addVar(vtype=GRB.BINARY, name="tourney5")
# y6 = m.addVar(vtype=GRB.BINARY, name="tourney6")
# y7 = m.addVar(vtype=GRB.BINARY, name="tourney7")
# y8 = m.addVar(vtype=GRB.BINARY, name="tourney8")
# y9 = m.addVar(vtype=GRB.BINARY, name="tourney9")
# y10 = m.addVar(vtype=GRB.BINARY, name="tourney10")
# y11= m.addVar(vtype=GRB.BINARY, name="tourney11")
# y12 = m.addVar(vtype=GRB.BINARY, name="tourney12")
# y13 = m.addVar(vtype=GRB.BINARY, name="tourney13")
# y14 = m.addVar(vtype=GRB.BINARY, name="tourney14")
# y15 = m.addVar(vtype=GRB.BINARY, name="tourney15")
# y16 = m.addVar(vtype=GRB.BINARY, name="tourney16")
# y17 = m.addVar(vtype=GRB.BINARY, name="tourney17")
# y18 = m.addVar(vtype=GRB.BINARY, name="tourney18")
# y19 = m.addVar(vtype=GRB.BINARY, name="tourney19")
# y20 = m.addVar(vtype=GRB.BINARY, name="tourney20")
# y21 = m.addVar(vtype=GRB.BINARY, name="tourney21")
# y22 = m.addVar(vtype=GRB.BINARY, name="tourney22")
# y23 = m.addVar(vtype=GRB.BINARY, name="tourney23")
# y24 = m.addVar(vtype=GRB.BINARY, name="tourney24")
# y25 = m.addVar(vtype=GRB.BINARY, name="tourney25")
# y26 = m.addVar(vtype=GRB.BINARY, name="tourney26")
# y27 = m.addVar(vtype=GRB.BINARY, name="tourney27")
# y28 = m.addVar(vtype=GRB.BINARY, name="tourney28")
# y29 = m.addVar(vtype=GRB.BINARY, name="tourney29")
# y30 = m.addVar(vtype=GRB.BINARY, name="tourney30")
# y31 = m.addVar(vtype=GRB.BINARY, name="tourney31")
# y32 = m.addVar(vtype=GRB.BINARY, name="tourney32")
# y33 = m.addVar(vtype=GRB.BINARY, name="tourney33")
# y34 = m.addVar(vtype=GRB.BINARY, name="tourney34")
# y35 = m.addVar(vtype=GRB.BINARY, name="tourney35")
# y36 = m.addVar(vtype=GRB.BINARY, name="tourney36")
# y37 = m.addVar(vtype=GRB.BINARY, name="tourney37")
# y38 = m.addVar(vtype=GRB.BINARY, name="tourney38")
# y39 = m.addVar(vtype=GRB.BINARY, name="tourney39")
# y40 = m.addVar(vtype=GRB.BINARY, name="tourney40")
# y41 = m.addVar(vtype=GRB.BINARY, name="tourney41")
# y42 = m.addVar(vtype=GRB.BINARY, name="tourney42")
# y43 = m.addVar(vtype=GRB.BINARY, name="tourney43")
# y44 = m.addVar(vtype=GRB.BINARY, name="tourney44")
# y45 = m.addVar(vtype=GRB.BINARY, name="tourney45")
# y46 = m.addVar(vtype=GRB.BINARY, name="tourney46")
# y47 = m.addVar(vtype=GRB.BINARY, name="tourney47")
# y48 = m.addVar(vtype=GRB.BINARY, name="tourney48")



m.setObjective(gp.quicksum(y[i] for i in range(1,48)), GRB.MAXIMIZE)

m.addConstr(3 * gp.quicksum(y[i] for i in range(1,48) >= 12))
m.setObjective(gp.quicksum(y[i] for i in range(1,48) >= 15), GRB.MAXIMIZE)

m.optimize()



Set parameter Username
Set parameter LicenseID to value 2700322
Academic license - for non-commercial use only - expires 2026-08-27
